# Final project

### Do what would make sense?

Month, itens and stores. Some itens can be more sold in some stores in specific months.

How can I found this?

01. To find sells by item, month and store. 
02. To find the stores that likely target store.
03. To do some encoding in categorical variables. 




In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
import numpy as np


In [2]:
from IPython.core.display import display, HTML


## Reading the data

In [3]:
#import data
df_i = pd.read_csv('items.csv')
df_s = pd.read_csv('shops.csv')
df_ic = pd.read_csv('item_categories.csv')
df_st = pd.read_csv('sales_train.csv.gz', compression='gzip')
df_st['dt'] = pd.to_datetime(df_st['date'],format='%d.%m.%Y')
df_ts = pd.read_csv('test.csv.gz', compression='gzip')
df_ss = pd.read_csv('sample_submission.csv.gz', compression='gzip')




In [4]:
def glimpse_data(df,title=''):
    first_row = '=' * 30 + title + 30 * '='
    
    
    str_html = (
        '<h2>%s</h2>'
        '<ul><li>Shape: %s</li>' 
        '<li>Head <br/>%s</li>'  
        '<li>Data types <br/>%s</li>'
        '<li>Describe <br/>%s</li>'
        '<li>Unique values <br/>%s</li>'
        '</ul>'
    %(str(title),
      str(df.shape),
      str(df.head().to_html()), 
      str(pd.DataFrame(df.dtypes,columns=['tipo']).to_html()),
      str(df.describe().to_html()),
      str(pd.DataFrame(df.nunique(),columns=['qtd']).to_html())
     )
    )
    display(HTML(str_html))


In [5]:
l = list(df_ic.item_category_name)
l_cat = l

for ind in range(1,8):
    l_cat[ind] = 'Access'

for ind in range(10,18):
    l_cat[ind] = 'Consoles'

for ind in range(18,25):
    l_cat[ind] = 'Consoles Games'

for ind in range(26,28):
    l_cat[ind] = 'phone games'

for ind in range(28,32):
    l_cat[ind] = 'CD games'

for ind in range(32,37):
    l_cat[ind] = 'Card'

for ind in range(37,43):
    l_cat[ind] = 'Movie'

for ind in range(43,55):
    l_cat[ind] = 'Books'

for ind in range(55,61):
    l_cat[ind] = 'Music'

for ind in range(61,73):
    l_cat[ind] = 'Gifts'

for ind in range(73,79):
    l_cat[ind] = 'Soft'


df_ic['cats'] = l_cat
df_item_cat = pd.merge(df_i, df_ic, on='item_category_id')[['item_id','cats']]


In [6]:
#A glimpse on dataframes
glimpse_data(df_ts, 'Test (df_ts)')
glimpse_data(df_ss, 'Sample submission (df_ss)')
glimpse_data(df_i,'Items (df_i)')
glimpse_data(df_s, 'Shops (df_s)')
glimpse_data(df_ic, 'Items categories (df_ic)')
glimpse_data(df_st, 'Sales train (df_st)')



### Type of data in Sales train 
<ul>
    <li>date: date</li>
    <li>date_block_num: categorical</li>
    <li>shop_id: categorical</li>
    <li>item_id: categorical</li>
    <li>item_price: numerical</li>
    <li> item_cnt_day: numerical</li>
</ul>


In [7]:
import datetime


In [8]:
print(str(df_st.dt.min()) + ' until ' +str(df_st.dt.max()))
print(sorted(df_st.shop_id.unique()))

2013-01-01 00:00:00 until 2015-10-31 00:00:00
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59]


In [9]:
df = df_st.copy()

#A first work with data

df['day'] = df_st['dt'].dt.day
df['month'] = df_st['dt'].dt.month
df['year'] = df_st['dt'].dt.year
df['ttl_item'] = df_st['item_cnt_day']


In [10]:
df['ano_mes'] = df.year.astype(str) + '_' + df.month.astype(str).str.zfill(2)

In [11]:
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,dt,day,month,year,ttl_item,ano_mes
0,02.01.2013,0,59,22154,999.00,1.0,2013-01-02,2,1,2013,1.0,2013_01
1,03.01.2013,0,25,2552,899.00,1.0,2013-01-03,3,1,2013,1.0,2013_01
2,05.01.2013,0,25,2552,899.00,-1.0,2013-01-05,5,1,2013,-1.0,2013_01
3,06.01.2013,0,25,2554,1709.05,1.0,2013-01-06,6,1,2013,1.0,2013_01
4,15.01.2013,0,25,2555,1099.00,1.0,2013-01-15,15,1,2013,1.0,2013_01


In [12]:
df_month = df.groupby(['ano_mes','item_id','shop_id'])['ttl_item'].sum().reset_index()

In [13]:
print(df_month.shape)
df_month['shop_item'] = df_month.shop_id.astype(str).str.zfill(5) + '_' + df_month.item_id.astype(str).str.zfill(6)
df_month.head()

(1609124, 4)


,ano_mes,item_id,shop_id,ttl_item,shop_item
0,2013_01,19,25,1.0,00025_000019
1,2013_01,27,1,1.0,00001_000027
2,2013_01,27,2,1.0,00002_000027
3,2013_01,27,10,1.0,00010_000027
4,2013_01,27,19,1.0,00019_000027


In [14]:
tb = df_month.pivot(index='shop_item', columns='ano_mes', values='ttl_item')
tb.fillna(0,axis=1,inplace=True)
m = pd.melt(tb.reset_index(), id_vars=['shop_item'], value_vars=tb.columns[1:])
m.columns = ['shop_item','ano_mes','target']

In [15]:
m['shop_id'] = m['shop_item'].apply(lambda x: x.split('_')[0]).astype(int)
m['item_id'] = m['shop_item'].apply(lambda x: x.split('_')[1]).astype(int)
m['year'] = m['ano_mes'].apply(lambda x: x.split('_')[0]).astype(int)
m['month'] = m['ano_mes'].apply(lambda x: x.split('_')[1]).astype(int)
m['day'] = 1
m['dt'] = pd.to_datetime(m[['year','month','day']])
m = pd.merge(m,df_item_cat,on='item_id')

In [16]:
glimpse_data(m,'Data')

In [17]:
m.set_index('dt',inplace=True)

In [18]:
m.fillna(0,axis=1,inplace=True)
m['lag01'] = m.groupby('shop_item').shift(1)['target']
m['lag12'] = m.groupby('shop_item').shift(12)['target']
m['lag12'].fillna(m['target'].mean(),inplace=True)
m['lag01'].fillna(m['target'].mean(),inplace=True)


In [19]:
m[m.shop_item=='00003_000030'].head()

,shop_item,ano_mes,target,shop_id,item_id,year,month,day,cats,lag01,lag12
dt,,,,,,,,,,,
2013-02-01,00003_000030,2013_02,4.0,3,30,2013,2,1,Movie,0.251265,0.251265
2013-03-01,00003_000030,2013_03,5.0,3,30,2013,3,1,Movie,4.000000,0.251265
2013-04-01,00003_000030,2013_04,2.0,3,30,2013,4,1,Movie,5.000000,0.251265
2013-05-01,00003_000030,2013_05,2.0,3,30,2013,5,1,Movie,2.000000,0.251265
2013-06-01,00003_000030,2013_06,1.0,3,30,2013,6,1,Movie,2.000000,0.251265


### Adding data:
#### Value of max target: total, total by item, total by store, by month, by month and item, by month and store
#### Value of min target: total, total by item, total by store, by month, by month and item, by month and store

In [20]:
n = m.copy().reset_index()
n['max_target'] = max(m.target)
n['min_target'] = min(m.target)

def get_maximo_target(df_ogm,colunas,nm_nova_coluna):
    sel = colunas+['target']
    d = df_ogm[sel].groupby(colunas,as_index=False).max()
    d.columns = colunas+[nm_nova_coluna]
    return pd.merge(df_ogm,d,on=colunas)
n = get_maximo_target(n,['month'],'max_target__month')
n = get_maximo_target(n,['month','item_id'],'max_target__month__item_id')
n = get_maximo_target(n,['month','item_id','shop_id'],'max_target__month__item_id__shop_id')
n = get_maximo_target(n,['month','shop_id'],'max_target__month__shop_id')
n = get_maximo_target(n,['shop_id'],'max_target__shop_id')



In [21]:
def get_minimo_target(df_ogm,colunas,nm_nova_coluna):
    sel = colunas+['target']
    d = df_ogm[sel].groupby(colunas,as_index=False).min()
    d.columns = colunas+[nm_nova_coluna]
    return pd.merge(df_ogm,d,on=colunas)
n = get_minimo_target(n,['month'],'min_target__month')
n = get_minimo_target(n,['month','item_id'],'min_target__month__item_id')
n = get_minimo_target(n,['month','item_id','shop_id'],'min_target__month__item_id__shop_id')
n = get_minimo_target(n,['month','shop_id'],'min_target__month__shop_id')
n = get_minimo_target(n,['shop_id'],'min_target__shop_id')


In [22]:
m = n.copy()
m['target_clip'] = m.target.clip(0,20)

In [23]:
m.head()

,dt,shop_item,ano_mes,target,shop_id,item_id,year,month,day,cats,...,max_target__month__item_id,max_target__month__item_id__shop_id,max_target__month__shop_id,max_target__shop_id,min_target__month,min_target__month__item_id,min_target__month__item_id__shop_id,min_target__month__shop_id,min_target__shop_id,target_clip
0,2013-02-01,00000_000030,2013_02,31.0,0,30,2013,2,1,Movie,...,112.0,31.0,67.0,67.0,-4.0,0.0,0.0,0.0,0.0,20.0
1,2014-02-01,00000_000030,2014_02,0.0,0,30,2014,2,1,Movie,...,112.0,31.0,67.0,67.0,-4.0,0.0,0.0,0.0,0.0,0.0
2,2015-02-01,00000_000030,2015_02,0.0,0,30,2015,2,1,Movie,...,112.0,31.0,67.0,67.0,-4.0,0.0,0.0,0.0,0.0,0.0
3,2013-02-01,00000_000031,2013_02,11.0,0,31,2013,2,1,Movie,...,47.0,11.0,67.0,67.0,-4.0,0.0,0.0,0.0,0.0,11.0
4,2014-02-01,00000_000031,2014_02,0.0,0,31,2014,2,1,Movie,...,47.0,11.0,67.0,67.0,-4.0,0.0,0.0,0.0,0.0,0.0


In [24]:


t.set_index(['shop_item','dt'],inplace=True)
cols_mt = ['cats','shop_id','item_id']
for c in cols_mt:
    t[c + '_mean_target'] = (t.groupby(c)['target_clip'].transform('mean') *
                     t.groupby(c)['target_clip'].transform('count') + t.target_clip.mean() * 100 ) / (t.groupby(c)['target_clip'].transform('count') + 100)
t.to_csv('modeling_base.csv')
t.head()

index  ano_mes  target  shop_id  item_id  year  \
shop_item    dt                                                           
00000_000030 2013-02-01      0  2013_02    31.0        0       30  2013   
             2014-02-01      1  2014_02     0.0        0       30  2014   
             2015-02-01      2  2015_02     0.0        0       30  2015   
00000_000031 2013-02-01      3  2013_02    11.0        0       31  2013   
             2014-02-01      4  2014_02     0.0        0       31  2014   

                         month  day   cats     lag01         ...           \
shop_item    dt                                              ...            
00000_000030 2013-02-01      2    1  Movie  0.251265         ...            
             2014-02-01      2    1  Movie  0.000000         ...            
             2015-02-01      2    1  Movie  0.000000         ...            
00000_000031 2013-02-01      2    1  Movie  0.251265         ...            
             2014-02-01      2    1  Movie  0.000000         ...            

                         max_target__shop_id  min_target__month  \
shop_item    dt                                                   
00000_000030 2013-02-01                 67.0               -4.0   
             2014-02-01                 67.0               -4.0   
             2015-02-01                 67.0               -4.0   
00000_000031 2013-02-01                 67.0               -4.0   
             2014-02-01                 67.0               -4.0   

                         min_target__month__item_id  \
shop_item    dt                                       
00000_000030 2013-02-01                         0.0   
             2014-02-01                         0.0   
             2015-02-01                         0.0   
00000_000031 2013-02-01                         0.0   
             2014-02-01                         0.0   

                         min_target__month__item_id__shop_id  \
shop_item    dt                                                
00000_000030 2013-02-01                                  0.0   
             2014-02-01                                  0.0   
             2015-02-01                                  0.0   
00000_000031 2013-02-01                                  0.0   
             2014-02-01                                  0.0   

                         min_target__month__shop_id  min_target__shop_id  \
shop_item    dt                                                            
00000_000030 2013-02-01                         0.0                  0.0   
             2014-02-01                         0.0                  0.0   
             2015-02-01                         0.0                  0.0   
00000_000031 2013-02-01                         0.0                  0.0   
             2014-02-01                         0.0                  0.0   

                         target_clip  cats_mean_target  shop_id_mean_target  \
shop_item    dt                                                               
00000_000030 2013-02-01         20.0           0.14665             0.049137   
             2014-02-01          0.0           0.14665             0.049137   
             2015-02-01          0.0           0.14665             0.049137   
00000_000031 2013-02-01         11.0           0.14665             0.049137   
             2014-02-01          0.0           0.14665             0.049137   

                         item_id_mean_target  
shop_item    dt                               
00000_000030 2013-02-01             0.969343  
             2014-02-01             0.969343  
             2015-02-01             0.969343  
00000_000031 2013-02-01             0.727610  
             2014-02-01             0.727610  

[5 rows x 27 columns]